In [14]:
import pandas as pd
import numpy as np
import torch
import autogluon.tabular
import os

In [15]:

print(os.getcwd())

/Users/zhaohongbo/PycharmProjects/Pytorch_learning/VC


In [16]:
df=pd.read_csv('../data/investments_VC_clean.csv')

In [17]:
df.head()
drop_lst=['permalink','homepage_url']

In [18]:
df.drop(columns=drop_lst, inplace=True)

In [19]:
"""
Data Wrangler 生成的单元格。
"""
def clean_data(df):
    # 将缺少的值替换为列: 'name' 中的 "Unknown"
    df = df.fillna({'name': "Unknown"})
    # 将缺少的值替换为列: 'category_list' 中的 "Unknown"
    df = df.fillna({'category_list': "Unknown"})
    # 将缺少的值替换为列: ' market ' 中的 "Unknown"
    df = df.fillna({' market ': "Unknown"})
    # 将缺少的值替换为列: 'status' 中的 "Unknown"
    df = df.fillna({'status': "Unknown"})
    # 将缺少的值替换为列: ' funding_total_usd ' 中的 "0"
    df = df.fillna({' funding_total_usd ': " -   "})
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,name,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,#waywire,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,&TV Communications,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,'Rock' Your Paper,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,Tallinn,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,(In)Touch Network,|Electronics|Guides|Coffee|Restaurants|Music|iPhone|Apps|Mobile|iOS|E-Commerce|,Electronics,"15,00,000",operating,GBR,NaN,London,London,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-R- Ranch and Mine,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
"""
Data Wrangler 生成的单元格。
"""
def clean_data(df_clean):
    # 将缺少的值替换为列: 'country_code' 中的 "Unknown"
    df_clean = df_clean.fillna({'country_code': "Unknown"})
    # 将缺少的值替换为列: 'state_code', 'region', 'city' 中的 "Unknown"
    df_clean = df_clean.fillna({'state_code': "Unknown", 'region': "Unknown", 'city': "Unknown"})
    df['seed_missing'] = df['seed'].isnull().astype(int)
    # 从公式创建的列 'seed_missing'
    df_clean['fund_info_missing'] = df_clean['seed'].isnull().astype(int)
    # 将缺少的值替换为列: 'venture'、'undisclosed'和其他列16 中的 0
    df_clean = df_clean.fillna({'venture': 0, 'undisclosed': 0, 'convertible_note': 0, 'debt_financing': 0, 'angel': 0, 'grant': 0, 'private_equity': 0, 'post_ipo_equity': 0, 'post_ipo_debt': 0, 'secondary_market': 0, 'round_A': 0, 'round_B': 0, 'round_C': 0, 'round_D': 0, 'round_E': 0, 'round_F': 0, 'round_G': 0, 'round_H': 0})
    # 将缺少的值替换为列: 'equity_crowdfunding', 'product_crowdfunding', 'seed' 中的 0
    df_clean = df_clean.fillna({'equity_crowdfunding': 0, 'product_crowdfunding': 0, 'seed': 0})
    return df_clean

df_clean_1 = clean_data(df_clean.copy())
df_clean_1.head()

,name,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,...,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,fund_info_missing
0,#waywire,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,&TV Communications,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,'Rock' Your Paper,|Publishing|Education|,Publishing,"40,000",operating,EST,Unknown,Tallinn,Tallinn,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,(In)Touch Network,|Electronics|Guides|Coffee|Restaurants|Music|iPhone|Apps|Mobile|iOS|E-Commerce|,Electronics,"15,00,000",operating,GBR,Unknown,London,London,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,-R- Ranch and Mine,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
"""
Data Wrangler 生成的单元格
"""
def clean_data(df_clean_1):
    target_cols = [
    'seed',              # 种子轮融资
    'venture',           # 风险投资
    'equity_crowdfunding', # 股权众筹
    'undisclosed',       # 未披露融资
    'convertible_note',  # 可转换债券融资
    'debt_financing',    # 债务融资
    'angel',             # 天使轮融资
    'grant',             # 政府/机构赠款
    'private_equity',    # 私募股权投资
    'post_ipo_equity',   # 上市后股权融资
    'post_ipo_debt',     # 上市后债务融资
    'secondary_market',  # 二级市场交易
    'product_crowdfunding', # 产品众筹
    'round_A',           # A轮融资
    'round_B',           # B轮融资
    'round_C',           # C轮融资
    'round_D',           # D轮融资
    'round_E',           # E轮融资
    'round_F',           # F轮融资
    'round_G',           # G轮融资
    'round_H'            # H轮融资
]
    df_clean_1[target_cols] = df_clean_1[target_cols].apply(np.log1p)
    return df_clean_1

df_clean_2 = clean_data(df_clean_1.copy())
df_clean_2.head()